In [4]:
import numpy as np
import pandas as pd
import os
import matplotlib.image as mpimg
import cv2

In [5]:
for filename, i in zip(os.listdir("data/class_color"), range(10)):
    img = cv2.imread("data/class_color/" + filename)
  #  cv2.imwrite("colorseg" + str(i) + ".png", img)

for filename, i in zip(os.listdir("data/raw_images"), range(10)):
    img = cv2.imread("data/raw_images/" + filename)
  #  cv2.imwrite("raw" + str(i) +".png", img)
    

data = []
annot = []

for i in range(100):
    img = cv2.imread("colorseg" + str(i) + ".png")
    annot.append(img)
    img=cv2.imread("raw" + str(i) + ".png")
    data.append(img)

In [6]:
categories = pd.read_csv("C:/Users/Asus Gamer/Downloads/segmentation-20181013T121450Z-001/segmentation/categories.csv", sep=',')
categories.head()
print("Number of categories:", len(categories), "\n")


FileNotFoundError: File b'C:/Users/Asus Gamer/Downloads/segmentation-20181013T121450Z-001/segmentation/categories.csv' does not exist

In [ ]:
#number of dataset
print(len(data))

In [ ]:
subcat = []
no_subcat = 0
for row in categories.name:
    subcat.append(row)
no_subcat = len(subcat)

cat = []
for row in categories.category:
    cat.append(row)

catid = []
for row in categories.catId:
    catid.append(row)
no_cat = 1
act = catid[0]
for i in range(len(catid)):
    if catid[i]!=act:
        no_cat+=1
        act=catid[i]

col = []
for row in categories.color:
    c = row.replace(" ", "").split(',')
    rgb = []
    for i in c:
        rgb.append(int(i))
    col.append(rgb)

print("Subcategories and their representational colors [R, G, B]: \n")
for i in range(len(subcat)):
    print("%30s \t" % subcat[i], end ="")
    print(col[i])

print("\nSubcategories by their categories: \n")
act = cat[0]
print(cat[0] + ":")
for i in range(no_subcat):
    if cat[i] != act:
        print("\n" + cat[i] + ":")
        act=cat[i]
    print("\t\t"+subcat[i])

In [ ]:
img=cv2.resize(annot[0], (0,0), fx=0.5, fy=0.5)
cv2.imwrite('resized_0.png', img)